In [ ]:
#default_exp spec_lib.translate

In [ ]:
#export
import pandas as pd
import numpy as np
import tqdm
import typing

from peptdeep.utils import explode_multiple_columns

from alphabase.spectrum_library.library_base import SpecLibBase

In [ ]:
#export
#@numba.njit #(cannot use numba for pd.Series)
def create_modified_sequence(
    df_items:typing.Tuple, # must be ('sequence','mods','mod_sites')
    translate_mod_dict:dict=None,
    mod_sep='[]'
):
    '''
    Translate `(sequence, mods, mod_sites)` into a modified sequence. Used by `df.apply()`.
    For example, `('ABCDEFG','Mod1@A;Mod2@E','1;5')`->`_A[Mod1@A]BCDE[Mod2@E]FG_`.
    Args:
        df_items (List): must be `(sequence, mods, mod_sites)`
        translate_mod_dict (dict): A dict to map alpha modification names to other software
        mod_seq (str): '[]' or '()', default '[]'
    '''
    nterm = '_'
    cterm = '_'
    mod_seq = df_items[0]
    if df_items[1]:
        mods = df_items[1].split(';')
        mod_sites = [int(i) for i in df_items[2].split(';')]
        rev_order = np.argsort(mod_sites)[::-1]
        mod_sites = [mod_sites[rev_order[i]] for i in range(len(mod_sites))]
        mods = [mods[rev_order[i]] for i in range(len(mods))]
        if translate_mod_dict is not None:
            mods = [translate_mod_dict[mod] for mod in mods]
        for _site, mod in zip(mod_sites, mods):
            if _site > 0:
                mod_seq = mod_seq[:_site] + mod_sep[0]+mod+mod_sep[1] + mod_seq[_site:]
            elif _site == -1:
                cterm += mod_sep[0]+mod+mod_sep[1]
            elif _site == 0:
                nterm += mod_sep[0]+mod+mod_sep[1]
            else:
                mod_seq = mod_seq[:_site] + mod_sep[0]+mod+mod_sep[1] + mod_seq[_site:]
    return nterm + mod_seq + cterm

In [ ]:
df = pd.DataFrame()
df['sequence'] = ['ACDEFGHIK']*10
df['mods'] = ['ModNterm;ModB@G;ModCterm;ModA@C']*10
df['mod_sites'] = ['0;6;-1;2']*10
df[['sequence','mods','mod_sites']].apply(create_modified_sequence, axis=1)

0    _[ModNterm]AC[ModA@C]DEFG[ModB@G]HIK_[ModCterm]
1    _[ModNterm]AC[ModA@C]DEFG[ModB@G]HIK_[ModCterm]
2    _[ModNterm]AC[ModA@C]DEFG[ModB@G]HIK_[ModCterm]
3    _[ModNterm]AC[ModA@C]DEFG[ModB@G]HIK_[ModCterm]
4    _[ModNterm]AC[ModA@C]DEFG[ModB@G]HIK_[ModCterm]
5    _[ModNterm]AC[ModA@C]DEFG[ModB@G]HIK_[ModCterm]
6    _[ModNterm]AC[ModA@C]DEFG[ModB@G]HIK_[ModCterm]
7    _[ModNterm]AC[ModA@C]DEFG[ModB@G]HIK_[ModCterm]
8    _[ModNterm]AC[ModA@C]DEFG[ModB@G]HIK_[ModCterm]
9    _[ModNterm]AC[ModA@C]DEFG[ModB@G]HIK_[ModCterm]
dtype: object

In [ ]:
assert create_modified_sequence(('ACDEFGHIK','ModNterm;ModB@G;ModCterm;ModA@C','0;6;-1;2'))=='_[ModNterm]AC[ModA@C]DEFG[ModB@G]HIK_[ModCterm]'

In [ ]:
assert create_modified_sequence(
    ('ACDEFGHIK','ModNterm;ModB@G;ModCterm;ModA@C','0;6;-1;2'),
    {'ModNterm':'Mod(Nterm)', 'ModCterm':'Mod(Cterm)', 'ModA@C':'ModA(C)', 'ModB@G':'ModB(G)'},
    mod_sep='()'
) == '_(Mod(Nterm))AC(ModA(C))DEFG(ModB(G))HIK_(Mod(Cterm))'

In [ ]:
#export

def _get_frag_info_from_column_name(column:str):
    '''
    Only used when convert alphabase libraries into other libraries
    '''
    idx = column.rfind('_')
    frag_type = column[:idx]
    ch_str = column[idx+2:]
    charge = int(ch_str)
    if len(frag_type)==1:
        loss_type = 'noloss'
    else:
        idx = frag_type.find('_')
        loss_type = frag_type[idx+1:]
        frag_type = frag_type[0]
    return frag_type, loss_type, charge

def _get_frag_num(columns, rows, frag_len):
    frag_nums = []
    for r,c in zip(rows, columns):
        if c[0] in 'xyz':
            frag_nums.append(frag_len-r)
        else:
            frag_nums.append(r+1)
    return frag_nums

def merge_precursor_fragment_df(
    precursor_df:pd.DataFrame, 
    fragment_mz_df:pd.DataFrame, 
    fragment_inten_df:pd.DataFrame, 
    top_n_inten:int,
    frag_type_head:str='FragmentType',
    frag_mass_head:str='FragmengMz',
    frag_inten_head:str='RelativeIntensity',
    frag_charge_head:str='FragmentCharge',
    frag_loss_head:str='FragmentLossType',
    frag_num_head:str='FragmentNumber',
    verbose=True,
):
    '''
    Convert alphabase library into a single dataframe. 
    This method is not important, as it will be only 
    used by DiaNN, or spectronaut, or others
    '''
    df = precursor_df.copy()
    frag_columns = fragment_mz_df.columns.values.astype('U')
    frag_type_list = []
    frag_loss_list = []
    frag_charge_list = []
    frag_mass_list = []
    frag_inten_list = []
    frag_num_list = []
    if verbose:
        iters = tqdm.tqdm(df[['frag_start_idx','frag_end_idx']].values)
    else:
        iters = df[['frag_start_idx','frag_end_idx']].values
    for start, end in iters:
        intens = fragment_inten_df.iloc[start:end,:].values # is loc[start:end-1,:] faster?
        masses = fragment_mz_df.iloc[start:end,:].values
        sorted_idx = np.argsort(intens.reshape(-1))[-top_n_inten:][::-1]
        idx_in_df = np.unravel_index(sorted_idx, masses.shape)

        frag_len = end-start
        rows = np.arange(frag_len, dtype=np.int32)[idx_in_df[0]]
        columns = frag_columns[idx_in_df[1]]

        frag_types, loss_types, charges = zip(
            *[_get_frag_info_from_column_name(_) for _ in columns]
        )

        frag_nums = _get_frag_num(columns, rows, frag_len)

        frag_type_list.append(frag_types)
        frag_loss_list.append(loss_types)
        frag_charge_list.append(charges)
        frag_mass_list.append(masses[idx_in_df])
        frag_inten_list.append(intens[idx_in_df])
        frag_num_list.append(frag_nums)
    
    df[frag_type_head] = frag_type_list
    df[frag_mass_head] = frag_mass_list
    df[frag_inten_head] = frag_inten_list
    df[frag_charge_head] = frag_charge_list
    df[frag_loss_head] = frag_loss_list
    df[frag_num_head] = frag_num_list

    return explode_multiple_columns(df, 
        [
            frag_type_head,
            frag_mass_head,
            frag_inten_head,
            frag_charge_head,
            frag_loss_head,
            frag_num_head
        ]
    )

    # try:
    #     return df.explode([
    #         frag_type_head,
    #         frag_mass_head,
    #         frag_inten_head,
    #         frag_charge_head,
    #         frag_loss_head,
    #         frag_num_head
    #     ])
    # except ValueError:
    #     # df.explode does not allow mulitple columns before pandas version 1.x.x.
    #     df = df.explode(frag_type_head)

    #     df[frag_mass_head] = _flatten(frag_mass_list)
    #     df[frag_inten_head] = _flatten(frag_inten_list)
    #     df[frag_charge_head] = _flatten(frag_charge_list)
    #     df[frag_loss_head] = _flatten(frag_loss_list)
    #     df[frag_num_head] = _flatten(frag_num_list)
    #     return df

mod_to_other_mod_dict = {
    "Carbamidomethyl@C": "Carbamidomethyl (C)",
    "Oxidation@M": "Oxidation (M)",
    "Phospho@S": "Phospho (STY)",
    "Phospho@T": "Phospho (STY)",
    "Phospho@Y": "Phospho (STY)",
    "GG@K": "GG (K)",
    "Acetyl@Protein N-term": "Acetyl (Protein N-term)",
}

from alphabase.constants.modification import MOD_DF
mod_to_unimod_dict = {}
for mod_name,unimod_id in MOD_DF[['name','unimod_id']].values:
    if unimod_id==-1 or unimod_id=='-1': continue
    mod_to_unimod_dict[mod_name] = f"UniMod:{unimod_id}"

def is_nterm_frag(frag_type:str):
    return frag_type[0] in 'abc'

def mask_fragment_intensity_by_mz_(
    fragment_mz_df:pd.DataFrame, 
    fragment_intensity_df:pd.DataFrame,
    min_frag_mz, max_frag_mz
):
    fragment_intensity_df.mask(
        (fragment_mz_df>max_frag_mz)|(fragment_mz_df<min_frag_mz),
        0, inplace=True
    )

def mask_fragment_intensity_by_frag_nAA(
    fragment_intensity_df:pd.DataFrame,
    precursor_df:pd.DataFrame,
    max_mask_frag_nAA
):
    if max_mask_frag_nAA <= 0: return
    b_mask = np.zeros(
        len(fragment_intensity_df), 
        dtype=bool
    )
    y_mask = b_mask.copy()
    for i_frag in range(max_mask_frag_nAA):
        b_mask[precursor_df.frag_start_idx.values+i_frag] = True
        y_mask[precursor_df.frag_end_idx.values-i_frag-1] = True
    
    masks = np.zeros(
        (
            len(fragment_intensity_df),
            len(fragment_intensity_df.columns)
        ), dtype=bool
    )
    for i,col in enumerate(fragment_intensity_df.columns.values):
        if is_nterm_frag(col):
            masks[:,i] = b_mask
        else:
            masks[:,i] = y_mask
    
    fragment_intensity_df.mask(
        masks, 0, inplace=True
    )

def speclib_to_single_df(
    speclib:SpecLibBase,
    *,
    translate_mod_dict:dict = None,
    keep_k_highest_fragments:int=12,
    min_frag_mz = 300,
    max_frag_mz = 2000,
    min_frag_intensity = 0.02,
    min_frag_nAA = 0,
    modloss='H3PO4',
    frag_type_head:str='FragmentType',
    frag_mass_head:str='FragmentMz',
    frag_inten_head:str='RelativeIntensity',
    frag_charge_head:str='FragmentCharge',
    frag_loss_head:str='FragmentLossType',
    frag_num_head:str='FragmentNumber',
    verbose = True,
)->pd.DataFrame:
    '''
    Convert alphabase library to diann (or Spectronaut) library dataframe
    This method is not important, as it will be only 
    used by DiaNN, or spectronaut, or others
    Args:
        translate_mod_dict (dict): a dict map modifications from alphabase to other software. Default: build-in `alpha_to_other_mod_dict`
        keep_k_highest_peaks (int): only keep highest fragments for each precursor. Default: 12
    Returns:
        pd.DataFrame: a single dataframe in the SWATH-like format
    '''
    df = pd.DataFrame()
    df['ModifiedPeptide'] = speclib._precursor_df[
        ['sequence','mods','mod_sites']
    ].apply(
        create_modified_sequence, 
        axis=1,
        translate_mod_dict=translate_mod_dict,
        mod_sep='[]'
    )

    df['frag_start_idx'] = speclib._precursor_df['frag_start_idx']
    df['frag_end_idx'] = speclib._precursor_df['frag_end_idx']
    
    df['PrecursorCharge'] = speclib._precursor_df['charge']
    if 'irt_pred' in speclib._precursor_df.columns:
        df['iRT'] = speclib._precursor_df['irt_pred']
    elif 'rt_pred' in speclib._precursor_df.columns:
        df['iRT'] = speclib._precursor_df['rt_pred']
    elif 'rt_norm' in speclib._precursor_df.columns:
        df['iRT'] = speclib._precursor_df['rt_norm']
    else:
        raise ValueError('precursor_df must contain the "rt_pred" or "rt_norm" column')

    if 'mobility_pred' in speclib._precursor_df.columns:
        df['IonMobility'] = speclib._precursor_df.mobility_pred
    elif 'mobility' in speclib._precursor_df.columns:
        df['IonMobility'] = speclib._precursor_df.mobility
    
    df['LabelModifiedSequence'] = df['ModifiedPeptide']
    df['StrippedPeptide'] = speclib._precursor_df['sequence']

    if 'precursor_mz' not in speclib._precursor_df.columns:
        speclib.calc_precursor_mz()
    df['PrecursorMz'] = speclib._precursor_df['precursor_mz']

    if 'proteins' in speclib._precursor_df.columns:
        df['ProteinName'] = speclib._precursor_df['proteins']
        df['UniprotID'] = df['ProteinName']
        df['ProteinGroups'] = df['ProteinName']

    if 'uniprot_ids' in speclib._precursor_df.columns:
        df['UniprotID'] = speclib._precursor_df['uniprot_ids']
        if 'ProteinName' not in df.columns:
            df['ProteinName'] = df['UniprotID']
            df['ProteinGroups'] = df['UniprotID']

    if 'genes' in speclib._precursor_df.columns:
        df['Genes'] = speclib._precursor_df['genes']

    if 'protein_group' in speclib._precursor_df.columns:
        df['ProteinGroups'] = speclib._precursor_df['protein_group']

    mask_fragment_intensity_by_mz_(
        speclib._fragment_mz_df,
        speclib._fragment_intensity_df,
        min_frag_mz, max_frag_mz
    )

    mask_fragment_intensity_by_frag_nAA(
        speclib._fragment_intensity_df,
        speclib._precursor_df,
        max_mask_frag_nAA=min_frag_nAA-1
    )

    df = merge_precursor_fragment_df(
        df,
        speclib._fragment_mz_df,
        speclib._fragment_intensity_df,
        top_n_inten=keep_k_highest_fragments,
        frag_type_head=frag_type_head,
        frag_mass_head=frag_mass_head,
        frag_inten_head=frag_inten_head,
        frag_charge_head=frag_charge_head,
        frag_loss_head=frag_loss_head,
        frag_num_head=frag_num_head,
        verbose=verbose
    )
    df = df[df['RelativeIntensity']>min_frag_intensity]
    df.loc[df[frag_loss_head]=='modloss',frag_loss_head] = modloss

    return df.drop(['frag_start_idx','frag_end_idx'], axis=1)

def speclib_to_swath_df(
    speclib:SpecLibBase,
    *,
    keep_k_highest_fragments:int=12,
    min_frag_mz = 300,
    max_frag_mz = 2000,
    min_frag_intensity = 0.02,
)->pd.DataFrame:
    speclib_to_single_df(
        speclib, 
        translate_mod_dict=mod_to_other_mod_dict,
        keep_k_highest_fragments=keep_k_highest_fragments,
        min_frag_mz = min_frag_mz,
        max_frag_mz = max_frag_mz,
        min_frag_intensity = min_frag_intensity,
    )

def translate_to_tsv(
    speclib:SpecLibBase,
    tsv_or_buf:str,
    *,
    keep_k_highest_fragments:int=12,
    min_frag_mz:float = 300,
    max_frag_mz:float = 2000,
    min_frag_intensity:float = 0.02,
    min_frag_nAA:int = 0,
    batch_size:int = 1000000,
    translate_mod_dict:dict = mod_to_unimod_dict,
):
    if isinstance(tsv_or_buf, str):
        f = open(tsv_or_buf, "w")
    else:
        f = tsv_or_buf
    _speclib = SpecLibBase()
    _speclib._fragment_intensity_df = speclib._fragment_intensity_df
    _speclib._fragment_mz_df = speclib._fragment_mz_df
    precursor_df = speclib._precursor_df
    for i in tqdm.tqdm(range(0, len(speclib._precursor_df), batch_size)):
        _speclib._precursor_df = precursor_df.iloc[i:i+batch_size]
        df = speclib_to_single_df(
            _speclib, translate_mod_dict=translate_mod_dict,
            keep_k_highest_fragments=keep_k_highest_fragments,
            min_frag_mz=min_frag_mz,
            max_frag_mz=max_frag_mz,
            min_frag_intensity=min_frag_intensity,
            min_frag_nAA=min_frag_nAA,
            verbose=False
        )
        df.to_csv(f, header=(i==0), sep="\t", index=False)
    if isinstance(tsv_or_buf, str):
        f.close()
        

In [ ]:
from alphabase.peptide.fragment import create_fragment_mz_dataframe
repeat = 10
charged_frag_types = ['b_z1','y_z1','y_modloss_z1']
precursor_df = pd.DataFrame({
    'sequence': ['ASGHCEWMKYR']*repeat+['ASGHCEWMAAR'],
    'mods': ['Acetyl@Protein N-term;Carbamidomethyl@C;Oxidation@M']*repeat+[''],
    'mod_sites': ['0;4;8']*repeat+[''],
    'nAA': 11,
    'NCE': 20,
    'instrument': 'QE',
    'rt_pred': 10,
    'charge': 2,
    'protein_name': 'unknown',
    'mobility_pred': 1,
})
precursor_df.loc[0,['mods','mod_sites']] = ['Phospho@S','2']
frag_mass_df = create_fragment_mz_dataframe(precursor_df, charged_frag_types)
frag_mass_df

,b_z1,y_z1,y_modloss_z1
0,72.044390,1376.527555,1278.550659
1,239.042750,1209.529195,0.000000
2,296.064213,1152.507732,0.000000
3,433.123125,1015.448820,0.000000
4,536.132310,912.439635,0.000000
...,...,...,...
105,585.208572,634.312978,0.000000
106,771.287885,448.233665,0.000000
107,902.328370,317.193180,0.000000
108,973.365484,246.156066,0.000000


In [ ]:
precursor_df

,sequence,mods,mod_sites,nAA,NCE,instrument,rt_pred,charge,protein_name,mobility_pred,frag_start_idx,frag_end_idx
0,ASGHCEWMKYR,Phospho@S,2,11,20,QE,10,2,unknown,1,0,10
1,ASGHCEWMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,10,2,unknown,1,10,20
2,ASGHCEWMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,10,2,unknown,1,20,30
3,ASGHCEWMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,10,2,unknown,1,30,40
4,ASGHCEWMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,10,2,unknown,1,40,50
5,ASGHCEWMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,10,2,unknown,1,50,60
6,ASGHCEWMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,10,2,unknown,1,60,70
7,ASGHCEWMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,10,2,unknown,1,70,80
8,ASGHCEWMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,10,2,unknown,1,80,90
9,ASGHCEWMKYR,Acetyl@Protein N-term;Carbamidomethyl@C;Oxidat...,0;4;8,11,20,QE,10,2,unknown,1,90,100


In [ ]:
spec_lib = SpecLibBase(charged_frag_types)
spec_lib._precursor_df = precursor_df
spec_lib._fragment_intensity_df = frag_mass_df
spec_lib._fragment_mz_df = frag_mass_df
df = speclib_to_single_df(spec_lib, min_frag_mz=300)
assert (df.FragmentMz>=300).all()
df = speclib_to_single_df(spec_lib, min_frag_mz=300,min_frag_nAA=3)
assert (df.FragmentNumber>=3).all()

100%|██████████| 11/11 [00:00<00:00, 3246.82it/s]


,ModifiedPeptide,PrecursorCharge,iRT,IonMobility,LabelModifiedSequence,StrippedPeptide,PrecursorMz,FragmentType,FragmentMz,RelativeIntensity,FragmentCharge,FragmentLossType,FragmentNumber
0,_AS[Phospho@S]GHCEWMKYR_,2,10,1,_AS[Phospho@S]GHCEWMKYR_,ASGHCEWMKYR,724.285972,y,1376.527555,1376.527555,1,noloss,10
0,_AS[Phospho@S]GHCEWMKYR_,2,10,1,_AS[Phospho@S]GHCEWMKYR_,ASGHCEWMKYR,724.285972,y,1278.550659,1278.550659,1,H3PO4,10
0,_AS[Phospho@S]GHCEWMKYR_,2,10,1,_AS[Phospho@S]GHCEWMKYR_,ASGHCEWMKYR,724.285972,b,1273.452993,1273.452993,1,noloss,10
0,_AS[Phospho@S]GHCEWMKYR_,2,10,1,_AS[Phospho@S]GHCEWMKYR_,ASGHCEWMKYR,724.285972,y,1209.529195,1209.529195,1,noloss,9
0,_AS[Phospho@S]GHCEWMKYR_,2,10,1,_AS[Phospho@S]GHCEWMKYR_,ASGHCEWMKYR,724.285972,y,1152.507732,1152.507732,1,noloss,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,_ASGHCEWMAAR_,2,10,1,_ASGHCEWMAAR_,ASGHCEWMAAR,609.760775,b,771.287885,771.287885,1,noloss,7
10,_ASGHCEWMAAR_,2,10,1,_ASGHCEWMAAR_,ASGHCEWMAAR,609.760775,y,763.355571,763.355571,1,noloss,6
10,_ASGHCEWMAAR_,2,10,1,_ASGHCEWMAAR_,ASGHCEWMAAR,609.760775,y,634.312978,634.312978,1,noloss,5
10,_ASGHCEWMAAR_,2,10,1,_ASGHCEWMAAR_,ASGHCEWMAAR,609.760775,b,585.208572,585.208572,1,noloss,6


In [ ]:
#hide
import tempfile
speclib_sdf = speclib_to_single_df(spec_lib)
with tempfile.TemporaryFile('w+') as f:
    translate_to_tsv(spec_lib, f, batch_size=2)
    f.seek(0)
    ddf = pd.read_csv(f, sep="\t")
assert len(ddf) == len(speclib_sdf)
assert ddf.StrippedPeptide.values[0] == speclib_sdf.StrippedPeptide.values[0]
assert ddf.StrippedPeptide.values[-1] == speclib_sdf.StrippedPeptide.values[-1]
assert ddf.PrecursorCharge.dtype==np.int

100%|██████████| 6/6 [00:00<00:00, 57.71it/s]
